In [90]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

In [91]:
df_path = r'C:\Users\livia\Desktop\UFG\Estudos\data_mining\UCMF_raw.csv'
df = pd.read_csv(df_path)

# Limpeza 

Limpieza inicial

In [92]:
# Colunas desnecessárias
df = df.drop(columns=['ID','Convenio'])
print("DataFrame após remoção de ID e Convenio:")
print(df.head())

DataFrame após remoção de ID e Convenio:
   Peso  Altura   IMC Atendimento        DN  IDADE   PULSOS  PA SISTOLICA  \
0   5.0      51  19.0    11/05/06  30/03/06   0.12  Normais           NaN   
1   3.5      50  14.0    25/05/05  19/05/05   0.02  Normais           NaN   
2   0.0       0   NaN    12/06/01  08/05/05  -4.05  Normais           NaN   
3   8.1      65  19.0    15/10/09  21/04/09   0.50  Normais           NaN   
4  40.0     151  18.0    14/01/08  14/08/95  12.89  Normais           NaN   

   PA DIASTOLICA            PPA NORMAL X ANORMAL      B2      SOPRO   FC  \
0            NaN  Não Calculado          Anormal  Normal  Sistólico  112   
1            NaN  Não Calculado          Anormal  Normal    ausente  128   
2            NaN  Não Calculado          Anormal  Normal  Sistólico   88   
3            NaN  Não Calculado          Anormal  Normal    ausente   92   
4            NaN  Não Calculado          Anormal  Normal    ausente   96   

            HDA 1 HDA2 SEXO            

In [93]:
# Padronização da coluna SEXO
condicoes = [
    df['SEXO'].str.lower().str.startswith('f', na=False),
    df['SEXO'].str.lower().str.startswith('m', na=False)
]
resultados = ['F', 'M']
df['SEXO_TRATADO'] = np.select(condicoes, resultados, default=df['SEXO'])
df.drop(columns=['SEXO'], inplace=True)

print(df.head())

   Peso  Altura   IMC Atendimento        DN  IDADE   PULSOS  PA SISTOLICA  \
0   5.0      51  19.0    11/05/06  30/03/06   0.12  Normais           NaN   
1   3.5      50  14.0    25/05/05  19/05/05   0.02  Normais           NaN   
2   0.0       0   NaN    12/06/01  08/05/05  -4.05  Normais           NaN   
3   8.1      65  19.0    15/10/09  21/04/09   0.50  Normais           NaN   
4  40.0     151  18.0    14/01/08  14/08/95  12.89  Normais           NaN   

   PA DIASTOLICA            PPA NORMAL X ANORMAL      B2      SOPRO   FC  \
0            NaN  Não Calculado          Anormal  Normal  Sistólico  112   
1            NaN  Não Calculado          Anormal  Normal    ausente  128   
2            NaN  Não Calculado          Anormal  Normal  Sistólico   88   
3            NaN  Não Calculado          Anormal  Normal    ausente   92   
4            NaN  Não Calculado          Anormal  Normal    ausente   96   

            HDA 1 HDA2                      MOTIVO1  \
0      Palpitacao  NaN  6

In [94]:
# Padronização da coluna SOPRO

df['SOPRO_LIMPO'] = df['SOPRO'].str.lower().str.strip()

condicoes_sopro = [
    # Se começar com 'sist', padronizar para 'Sistólico'
    df['SOPRO_LIMPO'].str.startswith('sist', na=False),
    # Se começar com 'aus', padronizar para 'Ausente'
    df['SOPRO_LIMPO'].str.startswith('aus', na=False),
    # Se começar com 'cont', padronizar para 'Contínuo'
    df['SOPRO_LIMPO'].str.startswith('cont', na=False)
]

# Definir os resultados padronizados
resultados_sopro = ['Sistólico', 'Ausente', 'Contínuo']

# Aplicar a padronização
df['SOPRO_TRATADO'] = np.select(
    condicoes_sopro,
    resultados_sopro,
    default=df['SOPRO'] 
)

df.drop(columns=['SOPRO', 'SOPRO_LIMPO'], inplace=True)

print(df.head())

   Peso  Altura   IMC Atendimento        DN  IDADE   PULSOS  PA SISTOLICA  \
0   5.0      51  19.0    11/05/06  30/03/06   0.12  Normais           NaN   
1   3.5      50  14.0    25/05/05  19/05/05   0.02  Normais           NaN   
2   0.0       0   NaN    12/06/01  08/05/05  -4.05  Normais           NaN   
3   8.1      65  19.0    15/10/09  21/04/09   0.50  Normais           NaN   
4  40.0     151  18.0    14/01/08  14/08/95  12.89  Normais           NaN   

   PA DIASTOLICA            PPA NORMAL X ANORMAL      B2   FC           HDA 1  \
0            NaN  Não Calculado          Anormal  Normal  112      Palpitacao   
1            NaN  Não Calculado          Anormal  Normal  128        Dispneia   
2            NaN  Não Calculado          Anormal  Normal   88   Assintomático   
3            NaN  Não Calculado          Anormal  Normal   92   Assintomático   
4            NaN  Não Calculado          Anormal  Normal   96  Dor precordial   

  HDA2                      MOTIVO1               

In [95]:
df = df.drop_duplicates()  #exclui as linhas duplicadas, mantendo apenas a primeira ocorrência.

In [96]:
# Tratando Peso = 0 e Altura = 0
indices_para_excluir = df[
    (df['Peso'] == 0) &
    (df['Altura'] == 0)
].index
df = df.drop(indices_para_excluir)
print(f"Linhas removidas por Peso=0 e Altura=0: {len(indices_para_excluir)}")

Linhas removidas por Peso=0 e Altura=0: 1594


Feature Engineering

In [97]:
# Correção de idade
df['Atendimento'] = pd.to_datetime(df['Atendimento'], format='%Y%m%d', errors='coerce')
df['DN'] = pd.to_datetime(df['DN'], format='%d/%m/%y', errors='coerce')

linhas_validas = df['DN'].notna() & df['Atendimento'].notna()

diferenca_em_dias = (df.loc[linhas_validas, 'Atendimento'] - df.loc[linhas_validas, 'DN']).dt.days
idade_calculada = (diferenca_em_dias / 365.25).round(2)
df.loc[linhas_validas, 'IDADE'] = idade_calculada

# Corrigir idades negativas e filtrar por público-alvo (0 a 20 anos)
df['IDADE'] = df['IDADE'].abs()
df = df[df['IDADE'] <= 20]
print(f"DataFrame após filtro de idade (máx. 20 anos). Total de objetos: {len(df)}")
df.drop(columns=['DN','Atendimento'], inplace=True)

DataFrame após filtro de idade (máx. 20 anos). Total de objetos: 9685


In [98]:
# Cálculo da Pressão de Pulso (PPA)
df['PPA'] = df['PA SISTOLICA'] - df['PA DIASTOLICA']
df.drop(columns=['PA SISTOLICA', 'PA DIASTOLICA'], inplace=True)

# Recálculo do IMC
df['IMC'] = df['Peso'] / (df['Altura']**2)
df.drop(columns=['Peso', 'Altura'], inplace=True)

print(df.head())

        IMC  IDADE   PULSOS   PPA NORMAL X ANORMAL      B2   FC  \
0  0.001922   0.12  Normais   NaN          Anormal  Normal  112   
1  0.001400   0.02  Normais   NaN          Anormal  Normal  128   
3  0.001917   0.50  Normais   NaN          Anormal  Normal   92   
4  0.001754  12.89  Normais   NaN          Anormal  Normal   96   
5  0.002045   5.89  Normais  45.0          Anormal  Normal   80   

            HDA 1 HDA2                      MOTIVO1  \
0      Palpitacao  NaN  6 - Suspeita de cardiopatia   
1        Dispneia  NaN  6 - Suspeita de cardiopatia   
3   Assintomático  NaN     5 - Parecer cardiológico   
4  Dor precordial  NaN     5 - Parecer cardiológico   
5   Assintomático  NaN     5 - Parecer cardiológico   

                               MOTIVO2 SEXO_TRATADO SOPRO_TRATADO  
0  6 - Palpitação/taquicardia/arritmia            M     Sistólico  
1                         6 - Dispnéia            M       Ausente  
3                                  NaN            M       Ause

In [99]:
contagem_nan_ppa = df['PPA'].isnull().sum()
print(f"O número de linhas com PPA NaN é: {contagem_nan_ppa}")

# porcentagem
total_linhas = len(df)
porcentagem_nan_ppa = (contagem_nan_ppa / total_linhas) * 100
print(f"A porcentagem de linhas com PPA NaN é: {porcentagem_nan_ppa:.2f}%")

O número de linhas com PPA NaN é: 5988
A porcentagem de linhas com PPA NaN é: 61.83%


In [100]:
df.drop(columns=['PPA'], inplace=True)

Tratando valores ausentes (NaN) e Outliers

In [101]:
# Imputação das colunas Categóricas (usando a MODA)
colunas_categoricas_imput = ['NORMAL X ANORMAL', 'B2', 'SOPRO_TRATADO', 'HDA 1', 'HDA2', 'MOTIVO1', 'MOTIVO2', 'SEXO_TRATADO']
for col in colunas_categoricas_imput:
    moda = df[col].mode().iloc[0]
    df.loc[:, col] = df[col].fillna(moda)

# Imputação das colunas Numéricas (usando a MEDIANA)
colunas_numericas_imput = ['PULSOS', 'FC', 'IMC'] 

for col in colunas_numericas_imput:
    
    if pd.api.types.is_numeric_dtype(df[col]):
        mediana = df[col].median()
        df.loc[:, col] = df[col].fillna(mediana)
        
    else:
        print(f"A coluna '{col}' não é numérica, imputando com a Moda.")
        moda = df[col].mode()[0]
        df.loc[:, col] = df[col].fillna(moda)

A coluna 'PULSOS' não é numérica, imputando com a Moda.
A coluna 'FC' não é numérica, imputando com a Moda.


In [102]:
#Tratamento de Outliers usando a mediana para limitar (Winsorização)
for col in ['PULSOS', 'FC']:
    
    df[col] = pd.to_numeric(df[col], errors='coerce') #substitui valores não numéricos por NaN
    
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - 3 * IQR
    limite_superior = Q3 + 3 * IQR

    # Substitui os   outliers por limites (Winsorização)
    df[col] = np.where(df[col] < limite_inferior, limite_inferior, df[col])
    df[col] = np.where(df[col] > limite_superior, limite_superior, df[col])

# Transformação de dados

Classificação IMC

In [103]:
percentis_meninos = {
    24: {'p5': 14.7, 'p85': 17.3, 'p95': 18.1},  # 2 anos (24 meses)
    60: {'p5': 13.8, 'p85': 16.6, 'p95': 17.4},  # 5 anos (60 meses)
    120: {'p5': 14.2, 'p85': 18.6, 'p95': 20.4}, # 10 anos (120 meses)
    180: {'p5': 16.1, 'p85': 23.2, 'p95': 25.9}, # 15 anos (180 meses)
    228: {'p5': 17.8, 'p85': 26.2, 'p95': 29.4}, # 19 anos (228 meses)
}
percentis_meninas = {
    24: {'p5': 14.3, 'p85': 17.1, 'p95': 18.0},  
    60: {'p5': 13.6, 'p85': 16.7, 'p95': 17.7},  
    120: {'p5': 14.0, 'p85': 18.9, 'p95': 21.1}, 
    180: {'p5': 16.3, 'p85': 24.1, 'p95': 27.2}, 
    228: {'p5': 17.6, 'p85': 26.5, 'p95': 30.1}, 
}

def classificar_imc_df(linha):
    sexo = linha['SEXO_TRATADO']
    idade_anos = linha['IDADE']
    imc = linha['IMC']
    
    if pd.isna(sexo) or pd.isna(idade_anos) or pd.isna(imc) or imc <= 0:
        return "Desconhecido" # Caso não consiga calcular após a imputação

    idade_total_meses = idade_anos * 12
    
    tabela_percentis = percentis_meninos if sexo == 'M' else percentis_meninas

    idade_chave_mais_proxima = min(tabela_percentis.keys(), key=lambda k: abs(k - idade_total_meses))
    percentis = tabela_percentis[idade_chave_mais_proxima]
    p5, p85, p95 = percentis['p5'], percentis['p85'], percentis['p95']
    
    if imc < p5:
        return "Abaixo do peso"
    elif p5 <= imc < p85:
        return "Peso normal"
    elif p85 <= imc < p95:
        return "Sobrepeso"
    else: # imc >= p95
        return "Obesidade"

df['classificacao_imc'] = df.apply(classificar_imc_df, axis=1)

In [ ]:
"""df['NORMAL X ANORMAL'] = df['NORMAL X ANORMAL'].astype(str).str.upper().str.strip()

def mapear_normal_anormal(valor):
    if 'NORMAL' in valor:
        return 0  # 0 = NORMAL
    # Para o restante (incluindo 'ANORMAL' e outros ruídos):
    return 1  # 1 = ANORMAL (Patologia)

df['ANORMAL_COD'] = df['NORMAL X ANORMAL'].apply(mapear_normal_anormal)

# 2. Verificar a Contagem de Classes após a padronização
print("\nVerificação de Classes após Padronização:")
print(df['ANORMAL_COD'].value_counts())

# 3. Remover a coluna original
df.drop(columns=['NORMAL X ANORMAL'], inplace=True)"""


Verificação de Classes após Padronização:
ANORMAL_COD
0    9687
1       1
Name: count, dtype: int64


In [104]:
# Codificação Categórica (Label e One-Hot Encoding)


# Coluna NORMAL X ANORMAL -> Codificação Binária (Label Encoding)
le = LabelEncoder()
df['ANORMAL_COD'] = le.fit_transform(df['NORMAL X ANORMAL'])
df.drop(columns=['NORMAL X ANORMAL'], inplace=True)


# Colunas Categóricas Nominais -> One-Hot Encoding
colunas_para_dummies = ['SEXO_TRATADO', 'B2', 'SOPRO_TRATADO', 'classificacao_imc'] 
df = pd.get_dummies(df, columns=colunas_para_dummies, prefix=colunas_para_dummies)


# Colunas de Motivo e HDA -> Remoção
df.drop(columns=['MOTIVO1', 'MOTIVO2', 'HDA 1', 'HDA2'], inplace=True)

colunas_para_escalar = ['IDADE', 'PULSOS', 'FC', 'IMC']

for col in colunas_para_escalar:      # Verifica se há NaN e preenche com a MEDIANA da coluna
  
    df[col] = pd.to_numeric(df[col], errors='coerce')
    
    df[col].replace([np.inf, -np.inf], np.nan, inplace=True) # Substitui inf e -inf por NaN
    
    mediana = df[col].median()
    df[col].fillna(mediana, inplace=True)


# Padronização (Escalonamento de Variáveis Numéricas Contínuas)
scaler = StandardScaler()
colunas_para_escalar = ['IDADE', 'PULSOS', 'FC', 'IMC']

df[colunas_para_escalar] = scaler.fit_transform(df[colunas_para_escalar])

print("\nDataFrame Final do Pré-processamento (Pronto para Mineração):")
print(df.head())
print(df.info())


DataFrame Final do Pré-processamento (Pronto para Mineração):
        IMC     IDADE  PULSOS        FC  ANORMAL_COD  SEXO_TRATADO_F  \
0  0.142330 -1.043077     NaN  0.944833            0           False   
1 -0.240906 -1.064596     NaN  1.902208            0           False   
3  0.138531 -0.961302     NaN -0.251886            0           False   
4  0.019049  1.704979     NaN -0.012542            0           False   
5  0.232648  0.198606     NaN -0.969917            0            True   

   SEXO_TRATADO_Indeterminado  SEXO_TRATADO_M  B2_Desdob fixo  \
0                       False            True           False   
1                       False            True           False   
3                       False            True           False   
4                       False            True           False   
5                       False           False           False   

   B2_Hiperfonética  B2_Normal  B2_Outro  B2_Única  SOPRO_TRATADO_Ausente  \
0             False       True     F

C:\Users\livia\AppData\Local\Temp\ipykernel_19212\49804080.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].replace([np.inf, -np.inf], np.nan, inplace=True) # Substitui inf e -inf por NaN
C:\Users\livia\AppData\Local\Temp\ipykernel_19212\49804080.py:27: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting valu

In [105]:
# Verificação de NaNs (Se houver NaNs após o escalonamento)
colunas_num = ['IMC', 'IDADE', 'PULSOS', 'FC']

# Impute novamente os NaNs
for col in colunas_num:
    df[col] = df[col].fillna(df[col].median())

# Verificação antes de continuar
print("Contagem de NaNs antes da modelagem:")
print(df[colunas_num].isnull().sum())

# Se a contagem for zero, o DataFrame está pronto!

Contagem de NaNs antes da modelagem:
IMC          0
IDADE        0
PULSOS    9685
FC           0
dtype: int64


c:\Users\livia\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [106]:
#Visto que a coluna pulsos está completamente vazia, vamos descartá-la.
df.drop(columns=['PULSOS'], inplace=True)

In [107]:
caminho_salvar = 'df_limpo_final.csv'
df.to_csv(caminho_salvar, index=False)
print(f"DataFrame limpo salvo em: {caminho_salvar}")

DataFrame limpo salvo em: df_limpo_final.csv
